# Evaluate end-to-end model

In this notebook, we measure the performance of the end-to-end model on the overall task of taking a Sudoku puzzle image and extracting all the cell labels.

In [1]:
from pathlib import Path
from tqdm import tqdm
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

from models.end_to_end.model import EndToEndModel
from models.end_to_end.dataset import get_data

In [2]:
model = EndToEndModel()

In [3]:
# For model selection, use "val".
# For final evaluation, use "test".

split = "test"

dataset = tf.data.Dataset.from_generator(lambda: get_data(split), output_types=(tf.float32, tf.int32))

In [4]:
accuracy = 0
count = 0

for image, labels in tqdm(dataset):
    count += 1
    labels_pred = model(image)
    accuracy += np.mean(labels_pred == labels)

accuracy /= count

print(f"Accuracy: {accuracy:%}")

80it [06:34,  4.93s/it]

Accuracy: 98.595679%


## Debug

In [ ]:
# Capture accuracy to use as a threshold on examples.

accuracy_threshold = accuracy

In [ ]:
# Find examples where the accuracy is worse than the average.

bad_examples = []

for image, labels in tqdm(dataset):
    labels_pred = model(image)
    accuracy = np.mean(labels_pred == labels)
    if accuracy < accuracy_threshold:
        bad_examples.append((image, labels, labels_pred, accuracy))

print(f"Found {len(bad_examples)} bad examples.")

In [ ]:
for i, example in enumerate(bad_examples):
    print(f"Accuracy on bad example {i:>2}: {example[-1]:>6.1%}")

In [ ]:
image, labels, labels_pred, accuracy = bad_examples[7]

In [ ]:
# Show original image

fig, ax = plt.subplots()

ax.set_title("Original image")
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
ax.imshow(image)

plt.show()

In [ ]:
import PIL.Image
import PIL.ImageDraw

def draw_keypoints(image, keypoints, color):
    image = tf.keras.preprocessing.image.array_to_img(image)
    draw = PIL.ImageDraw.Draw(image)
    draw.polygon(keypoints, outline=color, width=8)
    image = tf.keras.preprocessing.image.img_to_array(image) / 255.0
    return image

In [ ]:
# Show image with predicted keypoints

from models.end_to_end.model import relative_keypoints_to_absolute


keypoints = model.puzzle_localizer(image)
keypoints = relative_keypoints_to_absolute(image, keypoints)
image_with_predicted_keypoints = draw_keypoints(image, keypoints, "limegreen")

fig, ax = plt.subplots()

ax.set_title("Image with predicted keypoints")
ax.xaxis.set_visible(False)
ax.yaxis.set_visible(False)
ax.imshow(image_with_predicted_keypoints)

plt.show()